In [ ]:
import os

try:
    snakemake
except NameError:
    from snakemk_util import load_rule_args
    
    os.chdir('../..')
    
    snakemake = load_rule_args(
        snakefile = os.getcwd() + "/workflow/Snakefile",
        rule_name = 'tss_fantom_overlap',
        root = '..'
    )

In [ ]:
import pyranges as pr
from lapa.result import LapaTssResult